# **2.1 Feature Engineering**

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load data
movies = pd.read_csv('movie.csv')
ratings = pd.read_csv('rating.csv')
genome_scores = pd.read_csv('genome_scores.csv')
genome_tags = pd.read_csv('genome_tags.csv')

# Display the first few rows of each file to understand the structure
print("Movies:")
print(movies.head())
print("\nRatings:")
print(ratings.head())
print("\nGenome Scores:")
print(genome_scores.head())
print("\nGenome Tags:")
print(genome_tags.head())


In [ ]:
# Calculate average rating for each movie
movie_avg_ratings = ratings.groupby('movieId')['rating'].mean().reset_index()
movie_avg_ratings.rename(columns={'rating': 'ratings_avg'}, inplace=True)

# Merge the average ratings back to the movies DataFrame
movies = movies.merge(movie_avg_ratings, on='movieId', how='left')

print("Movies with average ratings:")
print(movies.head())


In [ ]:
# Split genres into binary features
genre_columns = movies['genres'].str.get_dummies(sep='|')

# Add the genre columns to the movies DataFrame
movies = pd.concat([movies, genre_columns], axis=1)

print("Movies with genre features:")
print(movies.head())

In [ ]:
# Merge genome_scores with genome_tags to get tag names
genome_data = genome_scores.merge(genome_tags, on='tagId', how='left')

# Find the most relevant tag for each movie
relevant_tags = genome_data.loc[genome_data.groupby('movieId')['relevance'].idxmax()]
relevant_tags = relevant_tags[['movieId', 'tag']].rename(columns={'tag': 'relevant_genome_tag'})

# Merge the relevant tags back to the movies DataFrame
movies = movies.merge(relevant_tags, on='movieId', how='left')

print("Movies with relevant tags:")
print(movies.head())

In [ ]:
# Calculate the number of ratings for each movie
movie_num_ratings = ratings.groupby('movieId')['rating'].count().reset_index()
movie_num_ratings.rename(columns={'rating': 'num_ratings'}, inplace=True)

# Merge the number of ratings back to the movies DataFrame
movies = movies.merge(movie_num_ratings, on='movieId', how='left')

print("Movies with number of ratings:")
print(movies.head())

In [ ]:
# Load tags data
tags = pd.read_csv('tag.csv')

# Find the most common tag for each movie
common_tags = tags.groupby('movieId')['tag'].agg(lambda x: x.value_counts().index[0]).reset_index()
common_tags.rename(columns={'tag': 'common_user_tag'}, inplace=True)

# Merge the common tags back to the movies DataFrame
movies = movies.merge(common_tags, on='movieId', how='left')

print("Movies with common user tags:")
print(movies.head())

In [ ]:
# Select relevant columns for clustering
final_data = movies[['movieId', 'ratings_avg', 'num_ratings', 'relevant_genome_tag', 'common_user_tag'] + list(genre_columns.columns)]

# Display the final data for clustering
print("Final Data for Clustering:")
print(final_data.head())


In [ ]:
# Add a new feature: Length of the movie title
movies['title_length'] = movies['title'].apply(len)

# Include this feature in the final dataset
final_data['title_length'] = movies['title_length']

In [ ]:
# Another new feature
movies['num_genres'] = movies['genres'].apply(lambda x: len(x.split('|')))

# Include this feature in the final dataset
final_data['num_genres'] = movies['num_genres']

In [ ]:
print(movies.head())

In [ ]:
# Another new feature
movies['avg_rating_per_genre'] = movies.groupby('genres')['ratings_avg'].transform('mean')

# Include this feature in the final dataset
final_data['avg_rating_per_genre'] = movies['avg_rating_per_genre']

We achieved more than eight features because we represented each genre as a separate binary feature. Since movies can belong to multiple genres, each unique genre contributes a separate column. Additionally, we included other derived features such as the number of ratings, average rating, most relevant genome tag, most common user tag, title length, and more. Together, these exceed eight features, allowing for a more robust clustering analysis.

# **2.2 Choose your features (variables)!**

**Standardize the data**

Clustering algorithms, such as K-means, are sensitive to the scale of features. If some features (e.g., num_ratings) have a larger range of values, they will dominate over features with smaller ranges (e.g., ratings_avg), which can distort the results.

We standardize numerical features, such as ratings_avg and num_ratings, to ensure all features contribute equally to the outcome.

We will use StandardScaler from the sklearn library to standardize the data (bringing the mean to 0 and the standard deviation to 1).

In [ ]:
from sklearn.preprocessing import StandardScaler

# Step 1: Select numeric features for normalization
numeric_features = ['ratings_avg', 'num_ratings']
categorical_features = ['relevant_genome_tag', 'common_user_tag']
genre_features = list(genre_columns)
additional_features = ['title_length', 'num_genres', 'avg_rating_per_genre']

# Combine all features for clustering
final_features = numeric_features + genre_features + additional_features
data_for_clustering = final_data[final_features]

# Step 2: Standardize numeric features
scaler = StandardScaler()
data_for_clustering[numeric_features] = scaler.fit_transform(data_for_clustering[numeric_features])
data_for_clustering[additional_features] = scaler.fit_transform(data_for_clustering[additional_features])
data_for_clustering[genre_features] = scaler.fit_transform(data_for_clustering[genre_features])

In [ ]:
data_for_clustering

**Dimensionality Reduction**

If we have too many features, it can increase computational complexity and make interpreting the results more challenging.
We will use PCA (Principal Component Analysis) to reduce the number of features while preserving the most important information. PCA will decrease the number of features, preparing the data for the next clustering task.

In [ ]:
print(data_for_clustering.isnull().sum())

In [ ]:
data_for_clustering['ratings_avg'].fillna(data_for_clustering['ratings_avg'].median(), inplace=True)
data_for_clustering['num_ratings'].fillna(data_for_clustering['num_ratings'].median(), inplace=True)
data_for_clustering['avg_rating_per_genre'].fillna(data_for_clustering['avg_rating_per_genre'].median(), inplace=True)


In [ ]:
print(data_for_clustering.isnull().sum())

In [ ]:
from sklearn.decomposition import PCA

# Step 3: Perform dimensionality reduction using PCA
pca = PCA(n_components=10)  # Reduce to 10 components for simplicity
pca_data = pca.fit_transform(data_for_clustering)

# Step 4: Analyze explained variance ratio
explained_variance = pca.explained_variance_ratio_
print(f"Explained variance ratio by each PCA component: {explained_variance}")
print(f"Cumulative explained variance: {explained_variance.cumsum()}")

# Step 5: Save transformed data
data_pca = pd.DataFrame(pca_data, columns=[f'PC{i+1}' for i in range(pca_data.shape[1])])

print("Transformed data after PCA:")
print(data_pca.head())

In [ ]:
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance_ratio) 

# Visualizing in a graph
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(cumulative_variance) + 1), cumulative_variance, marker='o', linestyle='--', label='Cumulative')
plt.bar(range(1, len(explained_variance_ratio) + 1), explained_variance_ratio, alpha=0.7, label='Explained Variance')

plt.title('Explained and cumulative variance by principal component', fontsize=14)
plt.xlabel('Number of principal components', fontsize=12)
plt.ylabel('Explained variace', fontsize=12)
plt.xticks(range(1, len(cumulative_variance) + 1))
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Compute the loadings
loadings = pd.DataFrame(
    pca.components_.T,  
    columns=[f"PC{i+1}" for i in range(pca.components_.shape[0])],
    index = data_for_clustering.columns
)

# Print loadings
print("Loadings (incidence of variables for each PC):")
print(loadings)
num_cp = 5
# Visualization of loadings fro the first 2 principal components
plt.figure(figsize=(10, 6))
loadings.iloc[:, :num_cp].plot(kind='bar', figsize=(10, 6))
plt.title("Loadings of variables for the first 2 PC")
plt.xlabel("Variables")
plt.ylabel("Loadings value")
plt.axhline(0, color='gray', linewidth=0.8, linestyle='--')
plt.legend([f"PC{i+1}" for i in range(num_cp)])  # Aggiorna automaticamente la legenda
plt.tight_layout()
plt.show()

After all this analysis we can choose the optimal number of principal components to use for our cluster analysis. In that case, a good number of components can me 5. The first reason is that between the 5th and the 6th compontent there is the last decrease of explained variance, even if its tiny in that contex is enough. Last reason is that with 5 PC we manage to have the 40% of the variance explained. It's a quite low value, but it's quite normal working with a very big number of features, most of which have no internal correlations. For this reason, when applying a dimensional reduction it is almost mandatory to lose a large part of the information. 5 principal components is a good compromise having an acceptable information retained, still obtaining a great dimensional reduction of the dataset, facilitating the work in cluster analysis.

# **2.3 Clustering**

**Optimal number of clusters**

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Elbow Method
inertia = []
silhouette_scores = []
cluster_range = range(2, 11)

for k in cluster_range:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(data_pca)
    inertia.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(data_pca, kmeans.labels_))

# Plot Elbow Method
plt.figure(figsize=(12, 6))
plt.plot(cluster_range, inertia, marker='o', label='Inertia')
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('Inertia')
plt.legend()
plt.show()

# Plot Silhouette Scores
plt.figure(figsize=(12, 6))
plt.plot(cluster_range, silhouette_scores, marker='o', label='Silhouette Score', color='orange')
plt.title('Silhouette Scores')
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.legend()
plt.show()


**Elbow Method:**
The inertia graph shows a sharp decrease in inertia values as the number of clusters increases, especially up to 4 clusters. After that, the rate of decrease slows significantly.
This suggests that the optimal number of clusters is likely around 4, as adding more clusters beyond this point does not significantly improve the model (inertia reduction becomes minimal).

**Silhouette Scores:**
The silhouette coefficient reaches its maximum at 2 clusters, then gradually decreases as the number of clusters increases.
However, having only 2 clusters might be too generalized and could miss important details in the data. Values for 3 and 4 clusters remain relatively high, indicating that these are good choices for the number of clusters.

**K-means**

In [ ]:
import numpy as np

# Initialize cluster centers randomly
def initialize_centers(data, k):
    np.random.seed(42)  # For reproducibility
    random_indices = np.random.choice(data.shape[0], size=k, replace=False)
    return data[random_indices]

# Assign points to the nearest cluster center
def assign_clusters(data, centers):
    distances = np.linalg.norm(data[:, np.newaxis] - centers, axis=2)
    return np.argmin(distances, axis=1)

# Update cluster centers
def update_centers(data, labels, k):
    new_centers = np.array([data[labels == i].mean(axis=0) for i in range(k)])
    return new_centers

# K-means algorithm using MapReduce logic
def kmeans(data, k, max_iters=100, tol=1e-4):
    centers = initialize_centers(data, k)
    for i in range(max_iters):
        labels = assign_clusters(data, centers)
        new_centers = update_centers(data, labels, k)

        # Check for convergence
        if np.all(np.abs(new_centers - centers) < tol):
            print(f"Converged at iteration {i}")
            break
        centers = new_centers
    return centers, labels

# Run K-means with chosen k
k = 13  # Set the number of clusters (from elbow method)
data = data_pca.values  # Convert PCA-transformed data to NumPy array

centers, labels = kmeans(data, k)

# Output results
print("Final cluster centers:")
print(centers)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Function to calculate Sum of Squared Distances (SSD)
def compute_ssd(data, centers, labels):
    ssd = 0
    for i, center in enumerate(centers):
        cluster_points = data[labels == i]
        ssd += np.sum((cluster_points - center) ** 2)
    return ssd

# Function to run the elbow method
def elbow_method(data, max_k=10):
    ssd_values = []
    k_values = range(1, max_k + 1)

    for k in k_values:
        print(f"Running K-means for k={k}...")
        centers, labels = kmeans(data, k)
        ssd = compute_ssd(data, centers, labels)
        ssd_values.append(ssd)

    return k_values, ssd_values

# Plotting the elbow curve
def plot_elbow_curve(k_values, ssd_values):
    plt.figure(figsize=(8, 6))
    plt.plot(k_values, ssd_values, marker='o', linestyle='-')
    plt.xlabel("Number of Clusters (k)")
    plt.ylabel("Sum of Squared Distances (SSD)")
    plt.title("Elbow Method for Optimal k")
    plt.grid()
    plt.show()

# PCA-transformed data
data = data_pca.values  # Ensure PCA data is a NumPy array

# Run elbow method
k_values, ssd_values = elbow_method(data, max_k=10)

# Plot the results
plot_elbow_curve(k_values, ssd_values)


In [ ]:
from collections import Counter
cluster_counts = Counter(labels)
print("Number of points in each cluster:")
for cluster_id, count in sorted(cluster_counts.items(), key=lambda item: item[1]):
    print(f"Cluster {cluster_id}: {count} points")

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Plot clusters in 3D
fig = plt.figure(figsize=(18, 6))

# First plot: PC1, PC2, PC3
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(pca_data[:, 0], pca_data[:, 1], pca_data[:, 2], c=labels, cmap='viridis')
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_zlabel('PC3')
ax1.set_title('Clustering 3D - PC1, PC2, PC3')

# Second plot: PC2, PC3, PC4
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(pca_data[:, 1], pca_data[:, 2], pca_data[:, 3], c=labels, cmap='viridis')
ax2.set_xlabel('PC2')
ax2.set_ylabel('PC3')
ax2.set_zlabel('PC4')
ax2.set_title('Clustering 3D - PC2, PC3, PC4')

# Third plot: PC3, PC4, PC5
ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(pca_data[:, 2], pca_data[:, 3], pca_data[:, 4], c=labels, cmap='viridis')
ax3.set_xlabel('PC3')
ax3.set_ylabel('PC4')
ax3.set_zlabel('PC5')
ax3.set_title('Clustering 3D - PC3, PC4, PC5')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].scatter(data_pca['PC1'], data_pca['PC2'], c=labels, cmap='viridis')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[1].scatter(data_pca['PC1'], data_pca['PC3'], c=labels, cmap='viridis')
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC3')
axes[2].scatter(data_pca['PC2'], data_pca['PC3'], c=labels, cmap='viridis')
axes[2].set_xlabel('PC2')
axes[2].set_ylabel('PC3')
plt.suptitle('Clustering 2D - Principal couples of PC')
plt.show()

**K-means++**

In [ ]:
def kmeans_pp(data, k, max_iter=300, tol=1e-4):
    n_samples, _ = data.shape
    centroids = []

    # Initialize centroids using K-means++
    first_idx = np.random.randint(0, n_samples)
    centroids.append(data[first_idx])

    for _ in range(1, k):
        distances = np.min([np.linalg.norm(data - c, axis=1) ** 2 for c in centroids], axis=0)
        probabilities = distances / distances.sum()
        next_idx = np.random.choice(range(n_samples), p=probabilities)
        centroids.append(data[next_idx])

    centroids = np.array(centroids)

    # Perform K-means clustering
    for _ in range(max_iter):
        distances = np.linalg.norm(data[:, np.newaxis] - centroids, axis=2)
        labels = np.argmin(distances, axis=1)
        new_centroids = np.array([data[labels == i].mean(axis=0) for i in range(k)])

        if np.linalg.norm(new_centroids - centroids) < tol:
            break

        centroids = new_centroids

    return centroids, labels

# Apply K-means++ to data_pca
k = 3
centroids_pp, labels_pp = kmeans_pp(data_pca.values, k)

# Analyze cluster sizes
from collections import Counter
cluster_counts_pp = Counter(labels_pp)
print("Cluster sizes with K-means++:")
for cluster_id, count in cluster_counts_pp.items():
    print(f"Cluster {cluster_id}: {count} points")

# Print cluster centers
print("\nCluster centers with K-means++:")
print(centroids_pp)


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(18, 6))

# First plot: PC1, PC2, PC3
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(pca_data[:, 0], pca_data[:, 1], pca_data[:, 2], c=labels_pp, cmap='viridis')
ax1.set_xlabel('PC1')
ax1.set_ylabel('PC2')
ax1.set_zlabel('PC3')
ax1.set_title('Clustering 3D - PC1, PC2, PC3')

# Second plot: PC2, PC3, PC4
ax2 = fig.add_subplot(132, projection='3d')
ax2.scatter(pca_data[:, 1], pca_data[:, 2], pca_data[:, 3], c=labels_pp, cmap='viridis')
ax2.set_xlabel('PC2')
ax2.set_ylabel('PC3')
ax2.set_zlabel('PC4')
ax2.set_title('Clustering 3D - PC2, PC3, PC4')

# Third plot: PC3, PC4, PC5
ax3 = fig.add_subplot(133, projection='3d')
ax3.scatter(pca_data[:, 2], pca_data[:, 3], pca_data[:, 4], c=labels_pp, cmap='viridis')
ax3.set_xlabel('PC3')
ax3.set_ylabel('PC4')
ax3.set_zlabel('PC5')
ax3.set_title('Clustering 3D - PC3, PC4, PC5')

plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
axes[0].scatter(data_pca['PC1'], data_pca['PC2'], c=labels_pp, cmap='viridis')
axes[0].set_xlabel('PC1')
axes[0].set_ylabel('PC2')
axes[1].scatter(data_pca['PC1'], data_pca['PC3'], c=labels_pp, cmap='viridis')
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC3')
axes[2].scatter(data_pca['PC2'], data_pca['PC3'], c=labels_pp, cmap='viridis')
axes[2].set_xlabel('PC2')
axes[2].set_ylabel('PC3')
plt.suptitle('Clustering 2D - Principal couples of PC')
plt.show()

The cluster sizes remain the same, indicating that both methods arrived at the same distribution of points across clusters.


The cluster centers for K-means++ differ from those of regular K-means. This is due to the difference in initialization methods. In K-means++, the initial centers are selected to minimize the likelihood of a poor choice of starting points, which typically leads to more stable and optimal results.
Regular K-means uses random initialization, which can lead to varying results across different runs.

**DBSCAN (Density-Based Spatial Clustering of Applications with Noise)**

1. The algorithm uses two main parameters:

* eps (epsilon): the maximum distance between two points for them to be considered in the same cluster.
* min_samples: the minimum number of points required within the eps radius for a point to be considered a "core" point.

2. Points are categorized into three types:

* Core points: have at least min_samples neighbors within the eps radius.

* Border points: are within the eps radius of a core point but do not themselves have enough neighbors to be considered core points.

* Noise points: do not belong to any cluster and are considered outliers.

3. Clusters are formed around core points. Border points are assigned to the nearest cluster. Noise points remain unclustered.

**Advantages of DBSCAN:**

* Works well with clusters of arbitrary shapes.
* Handles noise (outliers) effectively.
* Does not require specifying the number of clusters in advance.

**Disadvantages of DBSCAN:**

* Sensitive to the choice of eps and min_samples parameters.
* Struggles with high-dimensional data (due to increased computational complexity).

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import numpy as np

# Set DBSCAN parameters
eps_value = 0.5
min_samples_value = 5

# Apply DBSCAN to data_pca
dbscan = DBSCAN(eps=eps_value, min_samples=min_samples_value)
dbscan_labels = dbscan.fit_predict(data_pca.values)

# Analyze results
unique_clusters = np.unique(dbscan_labels)
print(f"Number of clusters (including noise): {len(unique_clusters)}")
print(f"Cluster labels: {unique_clusters}")

# Silhouette score
if len(unique_clusters) > 1:
    silhouette_avg = silhouette_score(data_pca.values[dbscan_labels != -1], dbscan_labels[dbscan_labels != -1])
    print(f"Average silhouette score (excluding noise): {silhouette_avg:.3f}")
else:
    print("Silhouette score not applicable (only one cluster).")

# Cluster sizes
from collections import Counter
cluster_counts = Counter(dbscan_labels)
for cluster_id, count in cluster_counts.items():
    print(f"Cluster {cluster_id}: {count} points")

In [ ]:
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import numpy as np

# Set DBSCAN parameters
eps_value = 1.0
min_samples_value = 5

# Apply DBSCAN to data_pca
dbscan = DBSCAN(eps=eps_value, min_samples=min_samples_value)
dbscan_labels = dbscan.fit_predict(data_pca.values)

# Analyze results
unique_clusters = np.unique(dbscan_labels)
print(f"Number of clusters (including noise): {len(unique_clusters)}")
print(f"Cluster labels: {unique_clusters}")

# Silhouette score
if len(unique_clusters) > 1:
    silhouette_avg = silhouette_score(data_pca.values[dbscan_labels != -1], dbscan_labels[dbscan_labels != -1])
    print(f"Average silhouette score (excluding noise): {silhouette_avg:.3f}")
else:
    print("Silhouette score not applicable (only one cluster).")

# Cluster sizes
from collections import Counter
cluster_counts = Counter(dbscan_labels)
for cluster_id, count in cluster_counts.items():
    print(f"Cluster {cluster_id}: {count} points")


### **2.4 Best Algorithm**

In [ ]:
import numpy as np
from sklearn.metrics import silhouette_score, davies_bouldin_score

# Inertia calculation
def calculate_inertia(data, labels, centers):
    inertia = 0
    for i, center in enumerate(centers):
        cluster_points = data[labels == i]
        inertia += np.sum((cluster_points - center) ** 2)
    return inertia

# K-means Results
print("K-means Results:")
data_array = data_pca.values  # Convert DataFrame to NumPy array
inertia_kmeans = calculate_inertia(data_array, labels, centers)
silhouette_kmeans = silhouette_score(data_array, labels)
davies_bouldin_kmeans = davies_bouldin_score(data_array, labels)
print(f"Inertia for K-means: {inertia_kmeans}")
print(f"Silhouette Score for K-means: {silhouette_kmeans}")
print(f"Davies-Bouldin Index for K-means: {davies_bouldin_kmeans}")

# K-means++ Results
print("\nK-means++ Results:")
inertia_kmeans_pp = calculate_inertia(data_array, labels_pp, centroids_pp)
silhouette_kmeans_pp = silhouette_score(data_array, labels_pp)
davies_bouldin_kmeans_pp = davies_bouldin_score(data_array, labels_pp)
print(f"Inertia for K-means++: {inertia_kmeans_pp}")
print(f"Silhouette Score for K-means++: {silhouette_kmeans_pp}")
print(f"Davies-Bouldin Index for K-means++: {davies_bouldin_kmeans_pp}")

# DBSCAN Results
print("\nDBSCAN Results:")
unique_clusters = np.unique(dbscan_labels)
if len(unique_clusters) > 1:  # Check if DBSCAN found multiple clusters
    silhouette_dbscan = silhouette_score(data_array[dbscan_labels != -1], dbscan_labels[dbscan_labels != -1])
    davies_bouldin_dbscan = davies_bouldin_score(data_array[dbscan_labels != -1], dbscan_labels[dbscan_labels != -1])
    print(f"Silhouette Score for DBSCAN: {silhouette_dbscan}")
    print(f"Davies-Bouldin Index for DBSCAN: {davies_bouldin_dbscan}")
else:
    silhouette_dbscan = None
    davies_bouldin_dbscan = None
    print("Silhouette Score and Davies-Bouldin Index are not applicable for DBSCAN (only one cluster detected).")

# Summary of Results
print("\nSummary of Clustering Metrics:")
print(f"K-means:\n - Inertia: {inertia_kmeans}\n - Silhouette Score: {silhouette_kmeans}\n - Davies-Bouldin Index: {davies_bouldin_kmeans}")
print(f"K-means++:\n - Inertia: {inertia_kmeans_pp}\n - Silhouette Score: {silhouette_kmeans_pp}\n - Davies-Bouldin Index: {davies_bouldin_kmeans_pp}")
if silhouette_dbscan is not None:
    print(f"DBSCAN:\n - Silhouette Score: {silhouette_dbscan}\n - Davies-Bouldin Index: {davies_bouldin_dbscan}")
else:
    print("DBSCAN metrics are not available due to insufficient clustering.")

# Cluster Size Distribution
from collections import Counter
print("\nCluster Size Distribution:")
print("K-means:", Counter(labels))
print("K-means++:", Counter(labels_pp))
print("DBSCAN:", Counter(dbscan_labels))


Clustering is a powerful tool to identify natural groupings within data. In this section, we evaluated the quality of clustering results using three algorithms: K-means, K-means++, and DBSCAN. Our goal was to determine the most suitable algorithm for the given dataset. Below are the detailed steps, metrics, and analysis:

---

#### **Optimal Number of Clusters (\( k_{opt} \))**
Based on the results from the Elbow Method and Silhouette Analysis in section 2.3, we determined that the optimal number of clusters for K-means and K-means++ is \( k_opt = 4 \).

For DBSCAN, the optimal parameters were identified as:
- \( eps = 1 )
- \( min\_samples = 5 )

These parameters were used to compare the three clustering methods.


#### **Metrics for Evaluating Clustering Quality**
We used the following metrics to assess the quality of clustering:
1. **Inertia**: The sum of squared distances from points to the cluster centers. Lower values indicate more compact clusters.
2. **Silhouette Score**: Measures how well clusters are separated and how similar points are within the same cluster. Higher values indicate better-defined clusters.
3. **Davies-Bouldin Index (DBI)**: Reflects the average similarity between each cluster and the one most similar to it. Lower values are better.
4. **Cluster Size Distribution**: Examines the balance in cluster sizes. Balanced clusters are generally desirable unless the data has inherent imbalances.


#### **Applying and Evaluating the Algorithms**

##### **1. K-means**
- **Number of clusters**: 4 (predefined as \( k_opt \)).
- **Metrics**:
  - **Inertia**: 769,611.93
  - **Silhouette Score**: 0.485
  - **Davies-Bouldin Index**: 0.633
- **Cluster sizes**:
  - Cluster 0: 1,364 points
  - Cluster 1: 10,074 points
  - Cluster 2: 10,954 points
  - Cluster 3: 4,886 points
- **Observations**:
  - Clusters are moderately well-separated with a balanced distribution.
  - Silhouette Score indicates moderately good separation and compactness.


##### **2. K-means++**
- **Number of clusters**: 4 (predefined as \( k_opt \)).
- **Metrics**:
  - **Inertia**: 782,496.54
  - **Silhouette Score**: 0.531
  - **Davies-Bouldin Index**: 0.582
- **Cluster sizes**:
  - Cluster 0: 7,200 points
  - Cluster 1: 16,388 points
  - Cluster 2: 658 points
  - Cluster 3: 3,032 points
- **Observations**:
  - K-means++ achieved better initialization compared to K-means, leading to slightly better compactness and separation.
  - Cluster sizes are more imbalanced compared to K-means.


##### **3. DBSCAN**
- **Number of clusters**: 31 (including noise).
- **Metrics**:
  - **Silhouette Score**: 0.302
  - **Davies-Bouldin Index**: 1.017
- **Cluster sizes**:
  - Dominant Cluster 0: 24,194 points
  - Noise (-1): 2,772 points
  - Remaining clusters: Mostly small, with less than 100 points each.
- **Observations**:
  - DBSCAN identified one dominant cluster and many smaller ones, indicating it is not well-suited for this dataset.
  - The negative silhouette score suggests overlapping or poorly separated clusters.
  - DBSCAN’s performance is sensitive to parameter selection.


#### **Comparing Results**

| Algorithm    | Inertia    | Silhouette Score | Davies-Bouldin Index | Number of Clusters | Dominant Cluster Size | Observations                                     |
|--------------|------------|------------------|-----------------------|---------------------|-----------------------|-------------------------------------------------|
| **K-means**  | 769,611.93 | 0.485            | 0.633                 | 4                   | 10,954 points         | Balanced clusters, moderately good separation. |
| **K-means++**| 782,496.54 | 0.531            | 0.582                 | 4                   | 16,388 points         | Faster convergence, more imbalanced clusters.  |
| **DBSCAN**   | N/A        | 0.302            | 1.017                 | 31                  | 24,194 points         | Poor separation, unsuitable for this dataset.  |


#### **Conclusion**
Based on the evaluation:
1. **K-means++** emerged as the most suitable algorithm, achieving the highest Silhouette Score and lowest Davies-Bouldin Index, indicating well-defined and compact clusters.
2. **K-means** performed slightly worse than K-means++ but produced more balanced clusters, making it a viable alternative depending on the application.
3. **DBSCAN** struggled with the dataset, identifying one dominant cluster and many small, poorly separated clusters. This method is unsuitable for this dataset without significant parameter tuning.

Overall, **K-means++** is recommended for its superior clustering quality, but **K-means** may be preferred if cluster balance is critical. DBSCAN is not recommended for this dataset.


#### Bonus Part

Top 3 by better pair of PC

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from itertools import combinations

# Numero di cluster
n_clusters = 3

# Prepara la matrice delle componenti principali
data_pca_np = data_pca.to_numpy()

# Per tutte le combinazioni di due componenti principali
scores = []  # Per memorizzare silhouette e Davies-Bouldin scores per ogni coppia
graphs_data = []  # Per memorizzare i dati per i grafici migliori

for pair in combinations(range(data_pca_np.shape[1]), 2):
    pc_x, pc_y = pair
    selected_data = data_pca_np[:, [pc_x, pc_y]]  # Seleziona due componenti

    # Esegui il K-means su queste due componenti
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans_labels = kmeans.fit_predict(selected_data)

    # Calcola i punteggi
    silhouette = silhouette_score(selected_data, kmeans_labels)
    davies_bouldin = davies_bouldin_score(selected_data, kmeans_labels)

    # Memorizza i risultati
    scores.append((pair, silhouette, davies_bouldin))
    graphs_data.append((pair, selected_data, kmeans_labels, kmeans.cluster_centers_, silhouette, davies_bouldin))

# Ordina le coppie per silhouette score (decrescente) e Davies-Bouldin index (crescente)
top_3_silhouette = sorted(scores, key=lambda x: x[1], reverse=True)[:3]
top_3_davies = sorted(scores, key=lambda x: x[2])[:3]

# Funzione per visualizzare i grafici
def plot_top_results(top_results, metric_name):
    for idx, (pair, selected_data, labels, centroids, silhouette, davies_bouldin) in enumerate(
            [graphs_data[i[0]] for i in top_results]):
        pc_x, pc_y = pair
        plt.figure(figsize=(8, 6))
        plt.scatter(selected_data[:, 0], selected_data[:, 1], c=labels, cmap='viridis', alpha=0.6, s=50)
        plt.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200, label='Centroids')
        plt.title(f'Top {idx + 1} ({metric_name}) - PC{pc_x + 1} vs PC{pc_y + 1}\n'
                  f'Silhouette: {silhouette:.2f}, Davies-Bouldin: {davies_bouldin:.2f}')
        plt.xlabel(f'PC{pc_x + 1}')
        plt.ylabel(f'PC{pc_y + 1}')
        plt.legend()
        plt.show()

# Visualizza i grafici delle top 3 coppie per ciascun indice
print("Top 3 pairs by Silhouette Score:")
plot_top_results([(scores.index(item), item[1]) for item in top_3_silhouette], "Silhouette Score")

print("Top 3 pairs by Davies-Bouldin Index:")
plot_top_results([(scores.index(item), item[2]) for item in top_3_davies], "Davies-Bouldin Index")


Or better if plots are side by side?

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score
from itertools import combinations

# Numero di cluster
n_clusters = 3

# Prepara la matrice delle componenti principali
data_pca_np = data_pca.to_numpy()

# Per tutte le combinazioni di due componenti principali
scores = []  # Per memorizzare silhouette e Davies-Bouldin scores per ogni coppia
graphs_data = []  # Per memorizzare i dati per i grafici migliori

for pair in combinations(range(data_pca_np.shape[1]), 2):
    pc_x, pc_y = pair
    selected_data = data_pca_np[:, [pc_x, pc_y]]  # Seleziona due componenti

    # Esegui il K-means su queste due componenti
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans_labels = kmeans.fit_predict(selected_data)

    # Calcola i punteggi
    silhouette = silhouette_score(selected_data, kmeans_labels)
    davies_bouldin = davies_bouldin_score(selected_data, kmeans_labels)

    # Memorizza i risultati
    scores.append((pair, silhouette, davies_bouldin))
    graphs_data.append((pair, selected_data, kmeans_labels, kmeans.cluster_centers_, silhouette, davies_bouldin))

# Ordina le coppie per silhouette score (decrescente) e Davies-Bouldin index (crescente)
top_3_silhouette = sorted(scores, key=lambda x: x[1], reverse=True)[:3]
top_3_davies = sorted(scores, key=lambda x: x[2])[:3]

# Funzione per visualizzare i grafici affiancati
def plot_top_results_side_by_side(top_results_silhouette, top_results_davies):
    fig, axes = plt.subplots(3, 2, figsize=(16, 18))  # 3 righe, 2 colonne
    
    for row, (silhouette_item, davies_item) in enumerate(zip(top_results_silhouette, top_results_davies)):
        for col, (item, metric_name, ax) in enumerate([
                (silhouette_item, "Silhouette Score", axes[row, 0]),
                (davies_item, "Davies-Bouldin Index", axes[row, 1])]):
            
            pair, selected_data, labels, centroids, silhouette, davies_bouldin = graphs_data[scores.index(item)]
            pc_x, pc_y = pair
            ax.scatter(selected_data[:, 0], selected_data[:, 1], c=labels, cmap='viridis', alpha=0.6, s=50)
            ax.scatter(centroids[:, 0], centroids[:, 1], c='red', marker='X', s=200, label='Centroids')
            ax.set_title(f'Top {row + 1} ({metric_name}) - PC{pc_x + 1} vs PC{pc_y + 1}\n'
                         f'Silhouette: {silhouette:.2f}, Davies-Bouldin: {davies_bouldin:.2f}')
            ax.set_xlabel(f'PC{pc_x + 1}')
            ax.set_ylabel(f'PC{pc_y + 1}')
            ax.legend()

    plt.tight_layout()
    plt.show()

# Visualizza i grafici delle top 3 coppie per ciascun indice
print("Visualizzazione dei grafici affiancati:")
plot_top_results_side_by_side(top_3_silhouette, top_3_davies)


Visualize iteration of kmeans for fixed steps or until convergence

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances_argmin

# Use only PC1 and PC2 for visualization
selected_data = data_pca[['PC8', 'PC10']].to_numpy()

# Parameters
n_clusters = 7
max_iter = 10

# Random initialization of centroids
rng = np.random.RandomState(42)
initial_centroids = selected_data[rng.choice(selected_data.shape[0], n_clusters, replace=False)]

centroids = initial_centroids
all_iterations = []
cluster_variances = []  # Store variances of individual clusters for each iteration

for iteration in range(max_iter):
    # Assign points to nearest cluster
    labels = pairwise_distances_argmin(selected_data, centroids)

    # Store cluster assignments for visualization
    all_iterations.append((centroids.copy(), labels.copy()))

    # Calculate variances for each cluster
    cluster_var = [
        np.sum((selected_data[labels == k] - centroids[k]) ** 2)
        for k in range(n_clusters)
    ]
    cluster_variances.append(cluster_var)

    # Recalculate centroids
    new_centroids = np.array([selected_data[labels == k].mean(axis=0) for k in range(n_clusters)])

    # Check for convergence (if centroids do not change)
    if np.all(centroids == new_centroids):
        print(f"Convergence reached at iteration {iteration + 1}")
        break

    centroids = new_centroids

# Visualization of clustering progression
colors = plt.cm.viridis(np.linspace(0, 1, n_clusters))  # Generate distinct colors for clusters

for iteration, (centroids_iter, labels_iter) in enumerate(all_iterations):
    # Create subplot for scatter plot and variance bar
    fig, ax = plt.subplots(2, 1, figsize=(8, 12), gridspec_kw={'height_ratios': [3, 1]})

    # Scatter plot
    ax[0].scatter(selected_data[:, 0], selected_data[:, 1], c=labels_iter, cmap='viridis', alpha=0.6, s=50)
    ax[0].scatter(centroids_iter[:, 0], centroids_iter[:, 1], c='red', marker='X', s=200, label='Centroids')
    ax[0].set_title(f'K-means Progression - Iteration {iteration + 1}')
    ax[0].set_xlabel('PC1')
    ax[0].set_ylabel('PC2')
    ax[0].legend()

    # Variance bar plot
    variances = cluster_variances[iteration]
    ax[1].bar(range(n_clusters), variances, color=colors, alpha=0.8)
    ax[1].set_title('Cluster Variance Breakdown')
    ax[1].set_xticks(range(n_clusters))
    ax[1].set_xticklabels([f'Cluster {k}' for k in range(n_clusters)])
    ax[1].set_ylabel('Variance (SSD)')
    ax[1].set_xlabel('Clusters')

    plt.tight_layout()
    plt.show()
